In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # show only errors

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Model

# Define your tasks and their respective number of classes
tasks = {
    'task1': 10,  # Replace with the number of classes for task 1
    'task2': 5,   # Replace with the number of classes for task 2
}

# Input shape and base MobileNetV2 model
input_shape = (224, 224, 3)  # Adjust the input shape according to your data
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Create task-specific branches with convolutional layers
outputs = {}
for task, num_classes in tasks.items():
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    
    # Add convolutional layers to the task-specific branch
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    
    x = Flatten()(x)
    
    # Add more convolutional layers or other layers as needed
    
    output = Dense(num_classes, activation='softmax', name=f'{task}_output')(x)
    outputs[task] = output

# Build the multi-task model
model = Model(inputs=base_model.input, outputs=outputs)

# Compile the model with individual loss functions for each task
losses = {task: 'categorical_crossentropy' for task in tasks}
metrics = {task: 'accuracy' for task in tasks}

model.compile(optimizer='adam', loss=losses, metrics=metrics)

# Print model summary
model.summary()


ValueError: Input 0 of layer conv2d_1 is incompatible with the layer: : expected min_ndim=4, found ndim=2. Full shape received: (None, 128)